In [ ]:
# J. RYU, Electron Microscopy and Spectroscopy Lab., Seoul National University
import numpy as np
from scipy import optimize
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
from matplotlib.colors import ListedColormap
import tkinter.filedialog as tkf
import hyperspy.api as hys
import ipywidgets as pyw

In [ ]:
def load_binary_4D_stack(img_adr, datatype, original_shape, final_shape, log_scale=False):
    stack = np.fromfile(img_adr, dtype=datatype)
    stack = stack.reshape(original_shape)
    print(stack.shape)
    if log_scale:
        stack = np.log(stack[:final_shape[0], :final_shape[1], :final_shape[2], :final_shape[3]])
    else:
        stack = stack[:final_shape[0], :final_shape[1], :final_shape[2], :final_shape[3]]
    
    print(stack.shape) 
    return stack

In [ ]:
# refer to https://scipy-cookbook.readthedocs.io/items/FittingData.html

def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape) # row, col
    x = (X*data).sum()/total # row
    y = (Y*data).sum()/total # col
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum()) # row
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum()) # col
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [ ]:
def gaussian_center(image, cbox_edge=0):
    y, x = np.indices(image.shape)
    if not cbox_edge:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
        
    else:
        cbox_outy = int(image.shape[0]/2 - cbox_edge/2)
        cbox_outx = int(image.shape[1]/2 - cbox_edge/2)
        center_box = image[cbox_outy:-cbox_outy, cbox_outx:-cbox_outx]
        fit_params = fitgaussian(center_box)
        (_, center_y, center_x, _, _) = fit_params
        center = [center_y+cbox_outy, center_x+cbox_outx]
        
    return center

In [ ]:
raw_adr = tkf.askopenfilename()
print(raw_adr)

In [ ]:
datatype = "float32"
o_shape = (256, 256, 130, 128)
f_shape = (256, 256, 128, 128)

In [ ]:
# load a data (4D data -> "f_stack")
f_stack = load_binary_4D_stack(raw_adr, datatype, o_shape, f_shape, log_scale=False)
print(np.max(f_stack))
print(np.min(f_stack))
print(np.mean(f_stack))
#f_stack = f_stack / np.max(f_stack)
#print(np.median(stack_4d))
f_shape = f_stack.shape

In [ ]:
# select an interesting area
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(np.sum(f_stack, axis=(2, 3)), cmap="gray")

def onselect(eclick, erelease):
    print('startposition: (%f, %f)' % (eclick.xdata, eclick.ydata))
    print('endposition  : (%f, %f)' % (erelease.xdata, erelease.ydata))

box = RectangleSelector(ax, onselect)
plt.show()

In [ ]:
# crop the data for the ROI
f_stack = f_stack[int(box.corners[1][0]):int(box.corners[1][2]), 
                                           int(box.corners[0][0]):int(box.corners[0][1])].copy()

f_shape = f_stack.shape
print(f_stack.shape)

In [ ]:
pacbed = np.mean(f_stack, axis=(0,1))
print(pacbed.shape)

In [ ]:
# find center position
center_pos = []
for i in range(f_shape[0]):
    for j in range(f_shape[1]):
        center_pos.append(gaussian_center(f_stack[i, j], cbox_edge=30))
        
center_pos = np.asarray(center_pos)
center_pos = np.reshape(center_pos, (f_shape[0], f_shape[1], -1))
print(center_pos.shape)
center_mean = np.mean(center_pos, axis=(0, 1))
print(center_mean)

# center distibution
plt.figure()
plt.hist(center_pos[:, :, 0].flatten(), bins=100, density=True, color="orange", label="center y position")
plt.hist(center_pos[:, :, 1].flatten(), bins=100, density=True, color="gray", alpha=0.5, label="center x position")
plt.grid()
plt.legend()
plt.show()

plt.figure()
plt.scatter(center_pos[:, :, 1], center_pos[:, :, 0], s=10.0, alpha=0.5)
plt.grid()
plt.scatter(center_mean[1], center_mean[0], s=20, c="red")
plt.xlabel("center x position", fontsize=20)
plt.ylabel("center y position", fontsize=20)
plt.show()

In [ ]:
# Gaussian-fitted center position
ct = center_mean.tolist()

In [ ]:
# just center of the diffraction images
y, x = np.indices(f_shape[2:])
ct = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0]).tolist()

In [ ]:
def fourd_roll_axis(stack):
    stack = np.rollaxis(np.rollaxis(stack, 2, 0), 3, 1)
    return stack

In [ ]:
def max_rad(shape, center=None):
    y, x = np.indices(shape)
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
    
    r = np.hypot(y - center[0], x - center[1])
    
    return np.max(r)

In [ ]:
def radial_indices(shape, radial_range, scale, center=None):
    y, x = np.indices(shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
    
    r = np.hypot(y - center[0], x - center[1]) * scale
    ri = np.ones(r.shape)
    
    if len(np.unique(radial_range)) > 1:
        ri[np.where(r <= radial_range[0])] = 0
        ri[np.where(r > radial_range[1])] = 0
        
    else:
        r = np.round(r)
        ri[np.where(r != round(radial_range[0]))] = 0
    
    return ri

In [ ]:
def angle_indices(shape, angle_range, center=None):
    y, x = np.indices(shape)
    y = y.astype("float64")
    x = x.astype("float64")
    
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
    
    y -= center[0]
    x -= center[1]
    
    angle_arr = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            angle_arr[i, j] = np.angle(complex(x[i, j], y[i, j]), deg=True)
            
    angle_arr = angle_arr + 180
    ri = np.ones(angle_arr.shape)
    
    if angle_range[1] > angle_range[0]:
        ri[np.where(angle_arr < angle_range[0])] = 0
        ri[np.where(angle_arr >= angle_range[1])] = 0
        
    elif angle_range[0] > angle_range[1]:
        ri *= 0.0
        ri[np.where(angle_arr < angle_range[1])] = 1
        ri[np.where(angle_arr >= angle_range[0])] = 1
        
    else:
        angle_arr = np.round(r)
        ri[np.where(angle_arr != round(angle_range[0]))] = 0
    
    return ri

In [ ]:
# radian (mrad) per pixel
#mrad_per_pixel = 1.0
mrad_per_pixel = 0.27045890769499215
radii = np.arange(max_rad(f_stack.shape[2:], center=ct)) * mrad_per_pixel
print("maximum angle = %.2f"%(radii[-1]))

In [ ]:
%matplotlib qt
fig, ax = plt.subplots(3, 3, figsize=(10, 10))

def virtual_detector(det1_in, det1_out, det2_in, det2_out, det3_in, det3_out):
    for i in range(3):
        for j in range(3):
            ax[i][j].cla()
    
    if det1_in > det1_out:
        det1 = [det1_out, det1_in]
        print("Warning! detector 1 (inner angle > outer angle)")
    else:
        det1 = [det1_in, det1_out]
        
    if det2_in > det2_out:
        det2 = [det2_out, det2_in]
        print("Warning! detector 2 (inner angle > outer angle)")
    else:
        det2 = [det2_in, det2_out]    
        
    if det3_in > det3_out:
        det3 = [det3_out, det3_in]
        print("Warning! detector 3 (inner angle > outer angle)")
    else:
        det3 = [det3_in, det3_out]
    
    det = []
    det.append(det1)
    det.append(det2)
    det.append(det3)
    
    ri = []
    for i in range(3):
        ri.append(radial_indices(f_shape[2:], det[i], mrad_per_pixel, center=ct))
    
    for i in range(3):
        ax[i][0].imshow(ri[i], cmap="gray")
        ax[i][0].axis("off")
        ax[i][0].set_title("(%.2f mrad, %.2f mrad)"%(det[i][0], det[i][1]), fontsize=20)
        
    for i in range(3):
        ax[i][1].imshow(pacbed, cmap="gray")
        ax[i][1].imshow(ri[i], cmap="Reds", alpha=0.3)
        ax[i][1].axis("off")
        
    for i in range(3):
        img_temp = np.sum(np.multiply(f_stack, ri[i]), axis=(2, 3))
        ax[i][2].imshow(img_temp, cmap="afmhot")
        ax[i][2].axis("off")
    
    fig.canvas.draw()
    fig.tight_layout()    

st = {"description_width": "initial"}
d1in = pyw.FloatText(value=1.0, description="D1 inner angle: ", style=st)
d1out = pyw.FloatText(value=5.0, description="D1 outer angle: ", style=st)

d2in = pyw.FloatText(value=5.0, description="D2 inner angle: ", style=st)
d2out = pyw.FloatText(value=10.0, description="D2 outer angle: ", style=st)

d3in = pyw.FloatText(value=10.0, description="D3 inner angle: ", style=st)
d3out = pyw.FloatText(value=15.0, description="D3 outer angle: ", style=st)

pyw.interact(virtual_detector, det1_in=d1in, det1_out=d1out, det2_in=d2in, det2_out=d2out, det3_in=d3in, det3_out=d3out)
fig.show()